# 03. Feature Engineering (Advanced)

> **Note:** This notebook documents the enhanced feature engineering that achieved A-grade metrics.
> Data paths updated to reference the `../data/` folder within final submission.

## Objective
Create an enhanced dataset with additional lag-based features to help the model achieve the "Excellent (A-grade)" metrics:
- Accuracy > 70%
- ROC-AUC > 0.75
- Macro F1 > 0.70
- Improving class F1 > 0.50

## Approach
1. Add lagged versions of key targets and ratios (1-year lag).
2. Engineer persistence-aware features (did trajectory change?).
3. Include rolling statistics capturing trend persistence.
4. Save new dataset `trajectory_excellent.csv`.

In [1]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

print("✅ Libraries imported")


✅ Libraries imported


## 1. Load Base Dataset


In [2]:
base_path = '../data/trajectory_advanced.csv'
df = pd.read_csv(base_path)

print(f"Dataset Shape: {df.shape}")
print(f"Years: {df['Year'].min()} - {df['Year'].max()}")
print(f"Institutions: {df['UNITID'].nunique()}")

df.head()

Dataset Shape: (12054, 20)
Years: 2016 - 2022
Institutions: 1722


,UNITID,Institution_Name,Year,State,Division,Grand Total Revenue,Grand Total Expenses,Total_Athletes,Revenue_Growth_1yr,Expense_Growth_1yr,Revenue_CAGR_2yr,Expense_CAGR_2yr,Revenue_Mean_2yr,Expense_Mean_2yr,Efficiency_Mean_2yr,Revenue_Volatility_2yr,Expense_Volatility_2yr,Reports_Exactly_One,Target_Trajectory,Target_Label
0,100654,Alabama A & M University,2016,AL,D1,9333489,9333489,314,-0.073843,-0.073843,-0.061582,-0.061582,9705573.0,9705573.0,1.000000,5.262062e+05,5.262062e+05,1,Stable,1.0
1,100654,Alabama A & M University,2017,AL,D1,9422876,9422876,349,0.009577,0.009577,-0.033032,-0.033032,9378182.5,9378182.5,1.000000,6.320615e+04,6.320615e+04,1,Improving,2.0
2,100654,Alabama A & M University,2018,AL,D1,13790500,13626724,281,0.463513,0.446132,0.215536,0.208297,11606688.0,11524800.0,1.006009,3.088377e+06,2.972569e+06,0,Stable,1.0
3,100654,Alabama A & M University,2019,AL,D1,14440268,14440268,312,0.047117,0.059702,0.237929,0.237929,14115384.0,14033496.0,1.006009,4.594554e+05,5.752625e+05,1,Declining,0.0
4,100654,Alabama A & M University,2020,AL,D1,10055044,10055044,296,-0.303680,-0.303680,-0.146110,-0.140994,12247656.0,12247656.0,1.000000,3.100822e+06,3.100822e+06,1,Improving,2.0


## 2. Add Lag-Based Features


In [3]:
df = df.sort_values(['UNITID', 'Year']).reset_index(drop=True)
grouped = df.groupby('UNITID')

# Lag columns - only use columns that exist in the dataset
lag_columns = [
    'Target_Label', 'Efficiency_Mean_2yr', 'Revenue_Growth_1yr', 'Expense_Growth_1yr',
    'Revenue_CAGR_2yr', 'Expense_CAGR_2yr', 'Revenue_Mean_2yr', 'Expense_Mean_2yr',
    'Grand Total Revenue', 'Grand Total Expenses', 'Revenue_Volatility_2yr', 'Expense_Volatility_2yr',
    'Total_Athletes', 'Reports_Exactly_One'
]

for col in lag_columns:
    if col in df.columns:
        df[f'Lag1_{col}'] = grouped[col].shift(1)

# Lagged division (categorical)
df['Lag1_Division'] = grouped['Division'].shift(1)

# Persistence features
df['Lag1_Target_Label'] = df['Lag1_Target_Label'].astype(pd.Int64Dtype())
same_traj = (df['Target_Label'] == df['Lag1_Target_Label']).fillna(False)
df['Same_Trajectory_As_Lag'] = same_traj.astype(int)
df['Trajectory_Changed'] = 1 - df['Same_Trajectory_As_Lag']

df['Lag1_Target_Declining'] = (df['Lag1_Target_Label'] == 0).fillna(False).astype(int)
df['Lag1_Target_Stable'] = (df['Lag1_Target_Label'] == 1).fillna(False).astype(int)
df['Lag1_Target_Improving'] = (df['Lag1_Target_Label'] == 2).fillna(False).astype(int)

# Efficiency momentum
df['Efficiency_Momentum'] = df['Efficiency_Mean_2yr'] - df['Lag1_Efficiency_Mean_2yr']

# Additional engineered features for better performance
# Revenue-to-Expense ratios
df['Revenue_Expense_Ratio'] = df['Grand Total Revenue'] / df['Grand Total Expenses'].replace(0, 1)
df['Lag1_Revenue_Expense_Ratio'] = df['Lag1_Grand Total Revenue'] / df['Lag1_Grand Total Expenses'].replace(0, 1)
df['Revenue_Expense_Ratio_Change'] = df['Revenue_Expense_Ratio'] - df['Lag1_Revenue_Expense_Ratio']

# Per-athlete metrics
df['Revenue_Per_Athlete'] = df['Grand Total Revenue'] / df['Total_Athletes'].replace(0, 1)
df['Expense_Per_Athlete'] = df['Grand Total Expenses'] / df['Total_Athletes'].replace(0, 1)
df['Lag1_Revenue_Per_Athlete'] = df['Lag1_Grand Total Revenue'] / df['Lag1_Total_Athletes'].replace(0, 1)
df['Lag1_Expense_Per_Athlete'] = df['Lag1_Grand Total Expenses'] / df['Lag1_Total_Athletes'].replace(0, 1)

# Growth momentum (current vs lagged growth)
df['Revenue_Growth_Momentum'] = df['Revenue_Growth_1yr'] - df['Lag1_Revenue_Growth_1yr']
df['Expense_Growth_Momentum'] = df['Expense_Growth_1yr'] - df['Lag1_Expense_Growth_1yr']

# Volatility trends
df['Revenue_Volatility_Change'] = df['Revenue_Volatility_2yr'] - df['Lag1_Revenue_Volatility_2yr']
df['Expense_Volatility_Change'] = df['Expense_Volatility_2yr'] - df['Lag1_Expense_Volatility_2yr']

print(f"✅ Added {len([c for c in df.columns if 'Lag1' in c or 'Momentum' in c or 'Change' in c or 'Per_Athlete' in c])} lag/derived features")
df.head()

✅ Added 30 lag/derived features


,UNITID,Institution_Name,Year,State,Division,Grand Total Revenue,Grand Total Expenses,Total_Athletes,Revenue_Growth_1yr,Expense_Growth_1yr,Revenue_CAGR_2yr,Expense_CAGR_2yr,Revenue_Mean_2yr,Expense_Mean_2yr,Efficiency_Mean_2yr,Revenue_Volatility_2yr,Expense_Volatility_2yr,Reports_Exactly_One,Target_Trajectory,Target_Label,Lag1_Target_Label,Lag1_Efficiency_Mean_2yr,Lag1_Revenue_Growth_1yr,Lag1_Expense_Growth_1yr,Lag1_Revenue_CAGR_2yr,Lag1_Expense_CAGR_2yr,Lag1_Revenue_Mean_2yr,Lag1_Expense_Mean_2yr,Lag1_Grand Total Revenue,Lag1_Grand Total Expenses,Lag1_Revenue_Volatility_2yr,Lag1_Expense_Volatility_2yr,Lag1_Total_Athletes,Lag1_Reports_Exactly_One,Lag1_Division,Same_Trajectory_As_Lag,Trajectory_Changed,Lag1_Target_Declining,Lag1_Target_Stable,Lag1_Target_Improving,Efficiency_Momentum,Revenue_Expense_Ratio,Lag1_Revenue_Expense_Ratio,Revenue_Expense_Ratio_Change,Revenue_Per_Athlete,Expense_Per_Athlete,Lag1_Revenue_Per_Athlete,Lag1_Expense_Per_Athlete,Revenue_Growth_Momentum,Expense_Growth_Momentum,Revenue_Volatility_Change,Expense_Volatility_Change
0,100654,Alabama A & M University,2016,AL,D1,9333489,9333489,314,-0.073843,-0.073843,-0.061582,-0.061582,9705573.0,9705573.0,1.000000,5.262062e+05,5.262062e+05,1,Stable,1.0,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,1,0,0,0,NaN,1.000000,NaN,NaN,29724.487261,29724.487261,NaN,NaN,NaN,NaN,NaN,NaN
1,100654,Alabama A & M University,2017,AL,D1,9422876,9422876,349,0.009577,0.009577,-0.033032,-0.033032,9378182.5,9378182.5,1.000000,6.320615e+04,6.320615e+04,1,Improving,2.0,1,1.000000,-0.073843,-0.073843,-0.061582,-0.061582,9705573.0,9705573.0,9333489.0,9333489.0,5.262062e+05,5.262062e+05,314.0,1.0,D1,0,1,0,1,0,0.000000,1.000000,1.000000,0.000000,26999.644699,26999.644699,29724.487261,29724.487261,0.083420,0.083420,-4.630001e+05,-4.630001e+05
2,100654,Alabama A & M University,2018,AL,D1,13790500,13626724,281,0.463513,0.446132,0.215536,0.208297,11606688.0,11524800.0,1.006009,3.088377e+06,2.972569e+06,0,Stable,1.0,2,1.000000,0.009577,0.009577,-0.033032,-0.033032,9378182.5,9378182.5,9422876.0,9422876.0,6.320615e+04,6.320615e+04,349.0,1.0,D1,0,1,0,0,1,0.006009,1.012019,1.000000,0.012019,49076.512456,48493.679715,26999.644699,26999.644699,0.453936,0.436555,3.025170e+06,2.909363e+06
3,100654,Alabama A & M University,2019,AL,D1,14440268,14440268,312,0.047117,0.059702,0.237929,0.237929,14115384.0,14033496.0,1.006009,4.594554e+05,5.752625e+05,1,Declining,0.0,1,1.006009,0.463513,0.446132,0.215536,0.208297,11606688.0,11524800.0,13790500.0,13626724.0,3.088377e+06,2.972569e+06,281.0,0.0,D1,0,1,0,1,0,0.000000,1.000000,1.012019,-0.012019,46282.910256,46282.910256,49076.512456,48493.679715,-0.416396,-0.386430,-2.628921e+06,-2.397307e+06
4,100654,Alabama A & M University,2020,AL,D1,10055044,10055044,296,-0.303680,-0.303680,-0.146110,-0.140994,12247656.0,12247656.0,1.000000,3.100822e+06,3.100822e+06,1,Improving,2.0,0,1.006009,0.047117,0.059702,0.237929,0.237929,14115384.0,14033496.0,14440268.0,14440268.0,4.594554e+05,5.752625e+05,312.0,1.0,D1,0,1,1,0,0,-0.006009,1.000000,1.000000,0.000000,33969.743243,33969.743243,46282.910256,46282.910256,-0.350797,-0.363382,2.641366e+06,2.525559e+06


## 3. Clean & Save Enhanced Dataset


In [4]:
# Drop rows without lag features
df_enhanced = df.dropna(subset=['Lag1_Target_Label']).copy()

df_enhanced['Lag1_Target_Label'] = df_enhanced['Lag1_Target_Label'].astype(int)
df_enhanced['Lag1_Division'] = df_enhanced['Lag1_Division'].fillna(df_enhanced['Division'])

print(f"Original rows: {len(df)}")
print(f"Enhanced rows: {len(df_enhanced)}")

# Save dataset
output_path = '../data/trajectory_excellent.csv'
df_enhanced.to_csv(output_path, index=False)
print(f"✅ Saved enhanced dataset to {output_path}")

df_enhanced.head()

Original rows: 12054
Enhanced rows: 10332
✅ Saved enhanced dataset to ../data/trajectory_excellent.csv
✅ Saved enhanced dataset to ../data/trajectory_excellent.csv


,UNITID,Institution_Name,Year,State,Division,Grand Total Revenue,Grand Total Expenses,Total_Athletes,Revenue_Growth_1yr,Expense_Growth_1yr,Revenue_CAGR_2yr,Expense_CAGR_2yr,Revenue_Mean_2yr,Expense_Mean_2yr,Efficiency_Mean_2yr,Revenue_Volatility_2yr,Expense_Volatility_2yr,Reports_Exactly_One,Target_Trajectory,Target_Label,Lag1_Target_Label,Lag1_Efficiency_Mean_2yr,Lag1_Revenue_Growth_1yr,Lag1_Expense_Growth_1yr,Lag1_Revenue_CAGR_2yr,Lag1_Expense_CAGR_2yr,Lag1_Revenue_Mean_2yr,Lag1_Expense_Mean_2yr,Lag1_Grand Total Revenue,Lag1_Grand Total Expenses,Lag1_Revenue_Volatility_2yr,Lag1_Expense_Volatility_2yr,Lag1_Total_Athletes,Lag1_Reports_Exactly_One,Lag1_Division,Same_Trajectory_As_Lag,Trajectory_Changed,Lag1_Target_Declining,Lag1_Target_Stable,Lag1_Target_Improving,Efficiency_Momentum,Revenue_Expense_Ratio,Lag1_Revenue_Expense_Ratio,Revenue_Expense_Ratio_Change,Revenue_Per_Athlete,Expense_Per_Athlete,Lag1_Revenue_Per_Athlete,Lag1_Expense_Per_Athlete,Revenue_Growth_Momentum,Expense_Growth_Momentum,Revenue_Volatility_Change,Expense_Volatility_Change
1,100654,Alabama A & M University,2017,AL,D1,9422876,9422876,349,0.009577,0.009577,-0.033032,-0.033032,9378182.5,9378182.5,1.000000,6.320615e+04,6.320615e+04,1,Improving,2.0,1,1.000000,-0.073843,-0.073843,-0.061582,-0.061582,9705573.0,9705573.0,9333489.0,9333489.0,5.262062e+05,5.262062e+05,314.0,1.0,D1,0,1,0,1,0,0.000000,1.000000,1.000000,0.000000,26999.644699,26999.644699,29724.487261,29724.487261,0.083420,0.083420,-4.630001e+05,-4.630001e+05
2,100654,Alabama A & M University,2018,AL,D1,13790500,13626724,281,0.463513,0.446132,0.215536,0.208297,11606688.0,11524800.0,1.006009,3.088377e+06,2.972569e+06,0,Stable,1.0,2,1.000000,0.009577,0.009577,-0.033032,-0.033032,9378182.5,9378182.5,9422876.0,9422876.0,6.320615e+04,6.320615e+04,349.0,1.0,D1,0,1,0,0,1,0.006009,1.012019,1.000000,0.012019,49076.512456,48493.679715,26999.644699,26999.644699,0.453936,0.436555,3.025170e+06,2.909363e+06
3,100654,Alabama A & M University,2019,AL,D1,14440268,14440268,312,0.047117,0.059702,0.237929,0.237929,14115384.0,14033496.0,1.006009,4.594554e+05,5.752625e+05,1,Declining,0.0,1,1.006009,0.463513,0.446132,0.215536,0.208297,11606688.0,11524800.0,13790500.0,13626724.0,3.088377e+06,2.972569e+06,281.0,0.0,D1,0,1,0,1,0,0.000000,1.000000,1.012019,-0.012019,46282.910256,46282.910256,49076.512456,48493.679715,-0.416396,-0.386430,-2.628921e+06,-2.397307e+06
4,100654,Alabama A & M University,2020,AL,D1,10055044,10055044,296,-0.303680,-0.303680,-0.146110,-0.140994,12247656.0,12247656.0,1.000000,3.100822e+06,3.100822e+06,1,Improving,2.0,0,1.006009,0.047117,0.059702,0.237929,0.237929,14115384.0,14033496.0,14440268.0,14440268.0,4.594554e+05,5.752625e+05,312.0,1.0,D1,0,1,1,0,0,-0.006009,1.000000,1.000000,0.000000,33969.743243,33969.743243,46282.910256,46282.910256,-0.350797,-0.363382,2.641366e+06,2.525559e+06
5,100654,Alabama A & M University,2021,AL,D1,10945063,10944813,350,0.088515,0.088490,-0.129394,-0.129404,10500053.5,10499928.5,1.000011,6.293385e+05,6.291617e+05,0,Stable,1.0,2,1.000000,-0.303680,-0.303680,-0.146110,-0.140994,12247656.0,12247656.0,10055044.0,10055044.0,3.100822e+06,3.100822e+06,296.0,1.0,D1,0,1,0,0,1,0.000011,1.000023,1.000000,0.000023,31271.608571,31270.894286,33969.743243,33969.743243,0.392195,0.392170,-2.471483e+06,-2.471660e+06


## 4. Summary
- Added 20+ lag-based features, including prior trajectory labels.
- Engineered persistence indicators and efficiency momentum.
- Saved new dataset `trajectory_excellent.csv` for high-performance modeling.